In [1]:
import numpy as np
import h5py
import trimesh
import matplotlib.pyplot as plt
from random import random, uniform
import dlib
from scipy.optimize import lsq_linear

In [27]:
%run initializers/pose_parameters.ipynb
%run initializers/shape_parameters.ipynb
%run features/basel_features/index_loader.ipynb
%run features/image_features/feature_extractor.ipynb
%run visualizers/model_mesh.ipynb
%run transformations/transformations.ipynb
%run basel_model/BaselModel.ipynb

In [46]:
class Face3DMM:
    """
    Face 3D Model Mesh construction from 2D image.
    """
    
    def __init__(self, basel_model_filepath,
                 basel_feature_points_path,
                 dlib_shape_predictor_filepath):
        """
        Initialize the Face3DMM constructor.
        
        Args:
          basel_model_filepath: the path to the basel model
          basel_feature_points_path: the path to the csv file containing the dlib to model points mapping
          dlib_shape_predictor_filepath: the path to the dlib predictor data model
        """
        
        
        # store dlib predictor path
        self.dlib_shape_predictor_filepath = dlib_shape_predictor_filepath
        
        self.baselModel = BaselModel(basel_model_filepath)
        
        # get indexes of model feature points and associated dlib feature point numbers, indexed at 1
        feature_point_indexes, self.dlib_points = get_feature_point_indexes(basel_feature_points_path)
        
        # extract only the feature points
        self.shape_mean_features = self.baselModel.getShapeMean(pointIndexes=feature_point_indexes)
        self.shape_pcaBasis_features = self.baselModel.getShapePCABasis(pointIndexes=feature_point_indexes)
        self.shape_pcaVariance = self.baselModel.getShapePCAVariance()
        
        # get the triangles for the model mesh
        self.triangles = self.baselModel.getModelMeshTriangles()
        
        # the number of points, features and components
        self.num_points = int(len(self.shape_mean_features)/3)
        self.num_features = int(len(feature_point_indexes)/3)
        self.num_components = len(self.shape_pcaVariance)
    
    def get_3DMM(self, image_path):
        """
        Gets a 3DMM for an image of a face.
        
        Args:
          image_path: the path to the image of a face
        
        Returns:
          A 3DMM for the face in the 2D image
        """
        
        # extract the dlib features
        dlib_features = extract_features(image_path, self.dlib_shape_predictor_filepath, self.dlib_points)
        
        # find optimal parameters
        #a, r, t, s = self.optimize(dlib_features, i=0, j=5000)
        %run optimizers/GradientDescent.ipynb
        %run optimizers/initializers/EstimationRefinementInitializer.ipynb
        optimizer = GradientDescent(
                mean=self.shape_mean_features,
                pcaBasis=self.shape_pcaBasis_features,
                pcaVariance=self.shape_pcaVariance,
                numIterations=5000,
                initializer=EstimationRefinementInitializer(numIterations=10))
        a, r, t, s = optimizer.optimize(dlib_features, 1)
        
        # construct and return the model mesh as a trimesh object
        return self.get_model_mesh(a)
    
    def get_projection_points(self, a, r=np.array([[1,0,0],[0,1,0],[0,0,1]]), t=np.array([0,0]).reshape(2,1), s=1., mode='all'):
        """
        Gets the 2D projections from the pca model weights.
        
        Args:
          a: the pca model weights        (c, 1)
          r: the rotation matrix          (3, 3)
          t: the translation matrix       (2, 1)
          s: the scale matrix             (1)
          mode: 'all' or 'features'
          where
            f: the number of features
            c: the number of components
        
        Returns:
          The 2D point projections         (2, n)
        """
        
        if mode.lower() == 'all':
            return project_2d(self.shape_mean, self.shape_pcaBasis, a, r, t, s)
        elif mode.lower() == 'features':
            return project_2d(self.shape_mean_features, self.shape_pcaBasis_features, a, r, t, s)
        else:
            raise Exception('model must be all or features')
            
    
    def get_model_mesh(self, a):
        """
        Gets a Trimesh object for the model mesh from the pca component weights
        
        Args:
          a: the pca model weights        (c, 1)
          where
            c: the number of components
        
        Returns:
          A Trimesh object for the model mesh
        """
        
        return get_model_mesh(model_point_combination(self.baselModel.getShapeMean(), self.baselModel.getShapePCABasis(), a), self.triangles)
    

In [47]:
x = Face3DMM(basel_model_filepath='./model2017-1_bfm_nomouth.h5',
             basel_feature_points_path='./feature_points.csv',
            dlib_shape_predictor_filepath='./shape_predictor_68_face_landmarks.dat')
mesh = x.get_3DMM('./test_images/test_image_2.jpeg')
mesh.show()

In [ ]:
points = x.get_projection_points(np.array([0 for _ in range(x.num_components)]).reshape((x.num_components, 1)), mode='features')
fig, ax = plt.subplots()
ax.scatter(points[0,:], points[1,:])

for i in range(points.shape[-1]):
    ax.annotate(i, (points[0,:][i], points[1,:][i]))